In [125]:
# Flattening players.json file into more usable data

In [126]:
# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

In [127]:
playersPath = os.path.join(PROJ_ROOT, "data", "interim", "players.json")
biosPath = os.path.join(PROJ_ROOT, "data", "interim", "yale_names_bios.csv.gz")

In [128]:
players = pd.read_json(playersPath)
bios = pd.read_csv(biosPath, compression="gzip")

In [129]:
players.head(5)

,B/T,City,Cl.,Events,High School,Hometown/High School,Hometown/Region,Ht.,Name,Name_link,No.,Pos.,Region,Weapon,Wt.,season,sport
0,,Manhasset,Freshman,None,Chaminade,"Manhasset, N.Y. / Chaminade","Manhasset, N.Y.",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,N.Y.,None,180,2008-09,Baseball
1,,Sunrise,Junior,None,Pine Crest School,"Sunrise, Fla. / Pine Crest School","Sunrise, Fla.",6-2,Joe Castaldi,/sports/m-basebl/mtt/castaldi_joe00.html,34,P,Fla.,None,190,2008-09,Baseball
10,,Okemos,Freshman,None,Okemos,"Okemos, Mich. / Okemos","Okemos, Mich.",6-2,Pat Ludwig,/sports/m-basebl/mtt/ludwig_pat00.html,26,P,Mich.,None,175,2008-09,Baseball
100,R/R,-,Freshman,None,-,- / -,-,,Bobby Cecere,/sports/m-basebl/2016-17/bios/cecere_bobby_a3p9,,RHP,-,None,,2016-17,Baseball
1000,None,Rapid City,Fr.,None,Stevens High School,"Rapid City, South Dakota / Stevens High School","Rapid City, South Dakota",None,Eric Sanderson,/sports/m-crewlt/2015-16/bios/sanderson_eric_edr3,None,None,South Dakota,None,None,2015-16,Crew (Lightweight)


In [130]:
bios.head(5)

,Unnamed: 0,Name,Bio
0,0,Ryan Brenner,"2009: Made 22 appearances, including 20 starts..."
1,1,Joe Castaldi,"2010: Team captain.2009: Made 12 appearances, ..."
2,2,Adam Straus-Goldfarb,2015-16 (Junior): Three seat of the fourth var...
3,3,Robert Wulbern,2012-13: Rowed in stroke-seat of the 2V at Eas...
4,4,Matt Coombs,2006 Backup DB who played in all but one game....


In [131]:
# Warning: Number of players is inconsistent across the 2 dataframes
bios = bios.dropna()

In [132]:
# Remove first column off bios
bios = bios.drop(["Unnamed: 0"], axis=1)

In [133]:
# Throw in the Bio column
players = players.merge(bios, how="left", on="Name")

In [134]:
# Add the headers
headersPath = os.path.join(PROJ_ROOT, "data", "interim", "yale_bio_parsed.csv.gz")
headers = pd.read_csv(headersPath)

In [135]:
headers.head()

,url,headers,synopsis
0,Row(_1=u'/sports/m-basebl/mtt/brenner_ryan00.h...,"{u'Position:': u'C', u'name': u'Ryan Brenner',...","Row(_1=u'\n2009: Made 22 appearances, includin..."
1,Row(_1=u'/sports/m-basebl/mtt/castaldi_joe00.h...,"{u'Position:': u'P', u'name': u'Joe Castaldi',...",Row(_1=u'\n2010: Team captain.\n2009: Made 12 ...
2,Row(_1=u'/sports/m-basebl/mtt/ludwig_pat00.html'),"{u'Hometown:': u'Princeton, N.J.', u'Major:': ...",Row(_1=u'\n2015-16 (Junior): Three\xc2\xa0seat...
3,Row(_1=u'/sports/m-basebl/2016-17/bios/cecere_...,"{u'name': u'Robert Wulbern', u'Year:': u'Jr.',...",Row(_1=u'\n2012-13:\xa0Rowed in stroke-seat of...
4,Row(_1=u'/sports/m-crewlt/2015-16/bios/sanders...,"{u'Position:': u'DB', u'Height:': u'6-0', u'Ye...","Row(_1=u""\n2006 Backup DB who played in all bu..."


In [136]:
headers["url"] = headers.apply(lambda x: x["url"][9:-2], axis=1)

In [137]:
# Rename columns to prepare for merge
headers.columns = ["Name_link", "headers", "synopsis"]

In [138]:
players = players.merge(headers[["Name_link", "headers"]], how="left", on="Name_link")

In [139]:
players.head()

,B/T,City,Cl.,Events,High School,Hometown/High School,Hometown/Region,Ht.,Name,Name_link,No.,Pos.,Region,Weapon,Wt.,season,sport,Bio,headers
0,,Manhasset,Freshman,None,Chaminade,"Manhasset, N.Y. / Chaminade","Manhasset, N.Y.",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,N.Y.,None,180,2008-09,Baseball,"2009: Made 22 appearances, including 20 starts...","{u'Position:': u'C', u'name': u'Ryan Brenner',..."
1,,Manhasset,Freshman,None,Chaminade,"Manhasset, N.Y. / Chaminade","Manhasset, N.Y.",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,N.Y.,None,180,2008-09,Baseball,"2010: Saw action in 43 games, making 41 starts...","{u'Position:': u'C', u'name': u'Ryan Brenner',..."
2,,Manhasset,Freshman,None,Chaminade,"Manhasset, N.Y. / Chaminade","Manhasset, N.Y.",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,N.Y.,None,180,2008-09,Baseball,"2009: Made 22 appearances, including 20starts....","{u'Position:': u'C', u'name': u'Ryan Brenner',..."
3,,Manhasset,Freshman,None,Chaminade,"Manhasset, N.Y. / Chaminade","Manhasset, N.Y.",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,N.Y.,None,180,2008-09,Baseball,2011: Appeared in 36 games with 34 starts … Ba...,"{u'Position:': u'C', u'name': u'Ryan Brenner',..."
4,,Sunrise,Junior,None,Pine Crest School,"Sunrise, Fla. / Pine Crest School","Sunrise, Fla.",6-2,Joe Castaldi,/sports/m-basebl/mtt/castaldi_joe00.html,34,P,Fla.,None,190,2008-09,Baseball,"2010: Team captain.2009: Made 12 appearances, ...","{u'Position:': u'P', u'name': u'Joe Castaldi',..."


In [140]:
# Preserve only the latest season
playersSorted = players.sort_values(["season"], ascending=False)
groupedFirst = playersSorted.groupby(by=["Name"]).first()
groupedLast = playersSorted.groupby(by=["Name"]).last()

In [141]:
groupedLast.iloc[2500]["season"][-2:]

u'09'

In [142]:
for i in range(groupedLast.shape[0]):
    first = groupedFirst.iloc[i]
    groupedLast.iloc[i]["season"] = groupedLast.iloc[i]["season"][:-2] + first["season"][-2:]

In [143]:
groupedLast.head(40)

,B/T,City,Cl.,Events,High School,Hometown/High School,Hometown/Region,Ht.,Name_link,No.,Pos.,Region,Weapon,Wt.,season,sport,Bio,headers
Name,,,,,,,,,,,,,,,,,,
A.J. Haase,None,Bonne Terre,Fr.,None,North County,"Bonne Terre, MO / North County","Bonne Terre, MO",6-4,/sports/m-footbl/2006-07/bios/haase_a.j.00.html,,TE,MO,None,245,2006-10,Football,2008: Started eight games at TE... LedYale end...,"{u'Position:': u'G', u'name': u'Josh Greenberg..."
AJ Edwards,None,Seattle,Fr.,None,South Kent School,"Seattle, Wash. / South Kent School","Seattle, Wash.",6-5,/sports/m-baskbl/2013-14/bios/edwards_aj_geat,25,G,Wash.,None,190,2013-17,Basketball,2013-14: Appeared in five games… Scored five p...,"{u'Position:': u'Pole Vault', u'name': u'Brend..."
Aarica West,None,Buena Park,Freshman,None,Vista Murrieta,"Buena Park, Calif. / Vista Murrieta","Buena Park, Calif.",5-10,/sports/w-baskbl/2009-10/bios/west_aarica,31,G/F,Calif.,None,None,2009-12,Basketball,Before Yale: Played four years scholasticallyi...,"{u'Hometown:': u'Concord, MA', u'High School:'..."
Aaron Ault,None,Altamonte Springs,Fr.,None,Lake Brantley,"Altamonte Springs, FL / Lake Brantley","Altamonte Springs, FL",6-3,/sports/m-footbl/2012-13/bios/ault_aaron_lcur,93,LB,FL,None,194,2012-16,Football,"2013: Played in all 10 games, mostly onspecial...","{u'Position:': u'DB', u'Height:': u'5-11', u'Y..."
Aaron Bosgang,None,Port Washington,Fr.,None,Paul D. Schreiber High School,"Port Washington, N.Y. / Paul D. Schreiber High...","Port Washington, N.Y.",None,/sports/m-crewlt/2016-17/bios/bosgang_aaron_uh2u,None,None,N.Y.,None,None,2016-17,Crew (Lightweight),Off the Water: President of National Honor Soc...,"{u'Position:': u'OL', u'Height:': u'6-6', u'Ye..."
Aaron Fuchs,None,Somerset West,Senior,None,Panel Vallei,"Somerset West, South Africa / Panel Vallei","Somerset West, South Africa",None,/sports/m-squash/2009-10/bios/fuchs_aaron,None,None,South Africa,None,None,2009-10,Squash,"Ibegan my squash adventure at the age of 14, m...","{u'name': u'Duncan Logie', u'Height:': u'6-6',..."
Aaron Greenberg,None,Eden Prairie,Fr.,Freestyle/Breaststroke/Backstroke,Eden Prairie High School,"Eden Prairie, Minn. / Eden Prairie High School","Eden Prairie, Minn.",None,/sports/m-swim/2013-14/bios/greenberg_aaron_ifc2,None,None,Minn.,None,None,2013-17,Swimming & Diving,Best Times: 50 Freestyle: 19.74 100 ...,"{u'Hometown:': u'Woodbridge, Conn.', u'Positio..."
Aaron Jones,None,"St. Catharines, Ont., Canada",Fr.,None,Governor Simcoe,"St. Catharines, Ont., Canada / Governor Simcoe","St. Catharines, Ont., Canada",None,/sports/m-crewlt/2013-14/bios/jones_aaron_g4f2,None,None,NOREGION?,None,None,2013-17,Crew (Lightweight),Before Yale: Rowed for St. Catharines Rowing C...,"{u'Position:': u'LB', u'Height:': u'6-3', u'Ye..."
Aaron Rodriguez,None,Cypress,Jr.,None,Cy-Fair,"Cypress, TX / Cy-Fair","Cypress, TX",5-5,/sports/m-soccer/2003-04/bios/rodriguez_aaron0...,7,Midfield,TX,None,135,2003-05,Soccer,"Rodriguez, who had an excellent spring season,...","{u'name': u""Hugh O'Cinneide"", u'Year:': u'Seni..."


In [144]:
processedPath = os.path.join(PROJ_ROOT, "data", "interim", "yale_bios_processed.csv")
groupedLast.to_csv(processedPath, encoding='utf-8')